In [1]:
import torch
import pandas as pd
import numpy as np

c:\Users\MohammadPC\anaconda3\envs\metats-env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
drop_columns = ['Unnamed: 0',
'unit.IRI_KEY',
'unit.SY',
'unit.GE',
'unit.VEND',
'unit.ITEM',
'price.IRI_KEY',
'price.SY',
'price.GE',
'price.VEND',
'price.ITEM',
'price.cate',
'F.IRI_KEY',
'F.SY',
'F.GE',
'F.VEND',
'F.ITEM',
'F.cate',
'D.IRI_KEY',
'D.SY',
'D.GE',
'D.VEND',
'D.ITEM',
'D.cate',
'holiday.IRI_KEY',
'holiday.SY',
'holiday.GE',
'holiday.VEND',
'holiday.ITEM',
'holiday.cate',]

# data:

In [8]:
def read_csv(i):
    # read each slots csv, create tensor of dimension:
    # (features, num_timeseries, lenght)
    dataset = []
    data = pd.read_csv(f"iri{i}.csv", usecols=lambda x: x not in drop_columns)

    cl = data.columns.to_list()
    clcat = ['unit.cate'] 
    clu = [c for c in cl if 'unit.1' in c]
    clp = [c for c in cl if 'price.1' in c]
    clh = [c for c in cl if 'holiday.1' in c]
    clf = [c for c in cl if 'F.' in c]
    cld = [c for c in cl if 'D.' in c]

    
    dataset.append(data[clu].values)
    #replace some inf values in price
    data_clp = data[clp].replace(np.inf, np.nan).interpolate()
    dataset.append(data_clp.values)
    dataset.append(data[clh].values)
    dataset.append(data[clf].values)
    dataset.append(data[cld].values)
    dataset = np.array(dataset)
    dataset = np.transpose(dataset,(0,1,2))
    return dataset , data[clcat].values

In [5]:
def normalize(d):
    # normalize unit and price for NNs
    norm = torch.nn.InstanceNorm1d(2)
    dd= norm(d[:2,:,:])
    return dd

In [6]:
def concat_slots(fist_slot, last_slot):
    # concat slots to create whole train/valid/test dataset 
    dataset = []
    catg = []
    for i in range(fist_slot, last_slot):
        tens , cat = read_csv(i)
        dataset.append(tens)
        catg.append(cat)

    catg = np.concatenate(catg,axis=0)
    dataset = np.concatenate(dataset,axis=1)
    d_troch = torch.Tensor(dataset)
    d_troch_normalized = normalize(d_troch)    
    return d_troch_normalized, catg

In [9]:
#datatrain
d_train, cat_train = concat_slots(1,11)     #5*83944*55, 83944*1
#datavalid
d_valid, cat_valid = concat_slots(11,13)    #15145
#datatest
d_test, cat_test = concat_slots(13,16)      #21049